In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 47.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=7383bf40887a5eb990207f05410268fe7d3c237492e539274db610483f05fe3f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext


In [3]:
from pyspark.sql import SparkSession
sc = SparkContext
spark = SparkSession.builder.appName('Recommendations 1M').getOrCreate()


In [67]:
movies = spark.read.csv('movies.csv', sep='::', header=None)
ratings = spark.read.csv('ratings.csv',sep='::',header=None)


In [68]:

ratings.describe()


DataFrame[summary: string, _c0: string, _c1: string, _c2: string, _c3: string]

In [69]:

movies.describe()


DataFrame[summary: string, _c0: string, _c1: string, _c2: string]

In [70]:
movies.limit(10).show()
movies.describe()


+---+--------------------+--------------------+
|_c0|                 _c1|                 _c2|
+---+--------------------+--------------------+
|  1|    Toy Story (1995)|Animation|Childre...|
|  2|      Jumanji (1995)|Adventure|Childre...|
|  3|Grumpier Old Men ...|      Comedy|Romance|
|  4|Waiting to Exhale...|        Comedy|Drama|
|  5|Father of the Bri...|              Comedy|
|  6|         Heat (1995)|Action|Crime|Thri...|
|  7|      Sabrina (1995)|      Comedy|Romance|
|  8| Tom and Huck (1995)|Adventure|Children's|
|  9| Sudden Death (1995)|              Action|
| 10|    GoldenEye (1995)|Action|Adventure|...|
+---+--------------------+--------------------+



DataFrame[summary: string, _c0: string, _c1: string, _c2: string]

In [71]:

ratings.limit(10).show()
ratings.describe()


+---+----+---+---------+
|_c0| _c1|_c2|      _c3|
+---+----+---+---------+
|  1|1193|  5|978300760|
|  1| 661|  3|978302109|
|  1| 914|  3|978301968|
|  1|3408|  4|978300275|
|  1|2355|  5|978824291|
|  1|1197|  3|978302268|
|  1|1287|  5|978302039|
|  1|2804|  5|978300719|
|  1| 594|  4|978302268|
|  1| 919|  4|978301368|
+---+----+---+---------+



DataFrame[summary: string, _c0: string, _c1: string, _c2: string, _c3: string]

In [30]:
print((ratings.count(), len(ratings.columns)))

(1000209, 4)


In [31]:
print((movies.count(), len(movies.columns)))

(3882, 3)


In [34]:
ratings = ratings.drop('_c3')
ratings.limit(10).show()


+---+----+---+
|_c0| _c1|_c2|
+---+----+---+
|  1|1193|  5|
|  1| 661|  3|
|  1| 914|  3|
|  1|3408|  4|
|  1|2355|  5|
|  1|1197|  3|
|  1|1287|  5|
|  1|2804|  5|
|  1| 594|  4|
|  1| 919|  4|
+---+----+---+



In [39]:
ratings = ratings.withColumn('UserID', col('UserID').cast('integer'))
ratings = ratings.withColumn('MovieID', col('MovieID').cast('integer'))
ratings = ratings.withColumn('Rating', col('rating').cast('float'))
ratings.limit(10).show()

+------+-------+------+
|UserID|MovieID|Rating|
+------+-------+------+
|     1|   1193|   5.0|
|     1|    661|   3.0|
|     1|    914|   3.0|
|     1|   3408|   4.0|
|     1|   2355|   5.0|
|     1|   1197|   3.0|
|     1|   1287|   5.0|
|     1|   2804|   5.0|
|     1|    594|   4.0|
|     1|    919|   4.0|
+------+-------+------+



In [37]:
ratings = ratings.drop('_c0')
ratings = ratings.drop('_c1')
ratings = ratings.drop('_c2')
ratings.show()

+------+-------+------+
|UserID|MovieID|Rating|
+------+-------+------+
|     1|   1193|   5.0|
|     1|    661|   3.0|
|     1|    914|   3.0|
|     1|   3408|   4.0|
|     1|   2355|   5.0|
|     1|   1197|   3.0|
|     1|   1287|   5.0|
|     1|   2804|   5.0|
|     1|    594|   4.0|
|     1|    919|   4.0|
|     1|    595|   5.0|
|     1|    938|   4.0|
|     1|   2398|   4.0|
|     1|   2918|   4.0|
|     1|   1035|   5.0|
|     1|   2791|   4.0|
|     1|   2687|   3.0|
|     1|   2018|   4.0|
|     1|   3105|   5.0|
|     1|   2797|   4.0|
+------+-------+------+
only showing top 20 rows



In [42]:
numerator = ratings.select("Rating").count()

# Count the number of distinct userIds and distinct movieIds
unique_users = ratings.select("UserID").distinct().count()
unique_movies = ratings.select("MovieID").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = unique_users * unique_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings data is ", "%.2f" % sparsity + "% empty.")
print("or sparsity =  ", "%.2f" % sparsity )


The ratings data is  95.53% empty.
or sparsity =   95.53


In [44]:
# Group data by userId, count ratings
UserID_pivot = ratings.groupBy("UserID").count().orderBy('count', ascending=False)
UserID_pivot.limit(10).show()



+------+-----+
|UserID|count|
+------+-----+
|  4169| 2314|
|  1680| 1850|
|  4277| 1743|
|  1941| 1595|
|  1181| 1521|
|   889| 1518|
|  3618| 1344|
|  2063| 1323|
|  1150| 1302|
|  1015| 1286|
+------+-----+



In [45]:
# Group data by userId, count ratings
MovieID_pivot = ratings.groupBy("MovieID").count().orderBy('count', ascending=False)
MovieID_pivot.limit(10).show()

+-------+-----+
|MovieID|count|
+-------+-----+
|   2858| 3428|
|    260| 2991|
|   1196| 2990|
|   1210| 2883|
|    480| 2672|
|   2028| 2653|
|    589| 2649|
|   2571| 2590|
|   1270| 2583|
|    593| 2578|
+-------+-----+



# Implementing ALS(Alternating Least Square) algorithm in Spark


In [46]:
# Import the required functions

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


In [47]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="UserID", 
          itemCol="MovieID",
          ratingCol="Rating",
          nonnegative = True,
          implicitPrefs = False,
          coldStartStrategy="drop")


In [48]:
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))


Num models to be tested:  16


In [49]:
# Build cross validation using CrossValidator
# numFolds=3 means the CrossValidator will create 3 different models.
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=2)

In [50]:
best_model=als.fit(train)


In [51]:
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())


**Best Model**
  Rank: 10
  MaxIter: 10
  RegParam: 0.1


In [52]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)


0.8717464157139776


In [54]:
test_predictions.show(5)


+------+-------+------+----------+
|UserID|MovieID|Rating|prediction|
+------+-------+------+----------+
|   148|     15|   5.0|  3.226206|
|   148|     17|   4.0| 3.8044214|
|   148|     60|   3.0| 3.3603506|
|   148|     89|   4.0| 3.3474038|
|   148|    208|   3.0| 3.0103722|
+------+-------+------+----------+
only showing top 5 rows



In [55]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

def somefunc1(value1):
  if   value1<3: 
      return 0
  else:
      return 1


def somefunc2(value1,value2):
  if   value1 == value2: 
      return 1
  else:
      return 0      
#convert to a UDF Function by passing in the function and return type of function

udfsomefunc1 = F.udf(somefunc1, IntegerType())
udfsomefunc2 = F.udf(somefunc2, IntegerType())
ratings_1 = test_predictions.withColumn("Rating_binary", udfsomefunc1("Rating"))
ratings_2 = ratings_1.withColumn("predictions_binary", udfsomefunc1("prediction"))
ratings_with_high_low = ratings_2.withColumn("Truth", udfsomefunc2("Rating_binary","predictions_binary"))
ratings_with_high_low.show()

+------+-------+------+----------+-------------+------------------+-----+
|UserID|MovieID|Rating|prediction|Rating_binary|predictions_binary|Truth|
+------+-------+------+----------+-------------+------------------+-----+
|   148|     15|   5.0|  3.226206|            1|                 1|    1|
|   148|     17|   4.0| 3.8044214|            1|                 1|    1|
|   148|     60|   3.0| 3.3603506|            1|                 1|    1|
|   148|     89|   4.0| 3.3474038|            1|                 1|    1|
|   148|    208|   3.0| 3.0103722|            1|                 1|    1|
|   148|    262|   4.0| 3.6733632|            1|                 1|    1|
|   148|    293|   3.0| 3.7762003|            1|                 1|    1|
|   148|    300|   2.0|  3.493458|            0|                 1|    0|
|   148|    316|   5.0| 3.8352897|            1|                 1|    1|
|   148|    332|   3.0| 2.8981779|            1|                 0|    0|
|   148|    338|   3.0|  3.204415|    

In [56]:
Perf_values = ratings_with_high_low.groupBy("Truth").count().orderBy('count', ascending=False)
Perf_values.limit(6).show()


+-----+------+
|Truth| count|
+-----+------+
|    1|163284|
|    0| 36406|
+-----+------+



In [57]:
a=list(Perf_values.select('Truth').toPandas()['Truth']) 
b=list(Perf_values.select('count').toPandas()['count'])

print(a)
print(b)


[1, 0]
[163284, 36406]


In [58]:
accuracy=b[0]/(b[0]+b[1])
print(accuracy*100)

81.76874154940157


# **Recommending Movies**
The final part of our code comes i.e. predicting the best movies for the user based on personalized choice and recommending the movies to the user.

In [59]:
# Generate n Recommendations for all users
recommendations = best_model.recommendForAllUsers(10)
recommendations.limit(10).show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+------+--------------------+
|UserID|     recommendations|
+------+--------------------+
|     1|[{572, 5.2933993}...|
|     3|[{572, 5.370448},...|
|     5|[{557, 4.643818},...|
|     6|[{572, 5.3602695}...|
|     9|[{572, 4.8109536}...|
|    12|[{572, 4.932078},...|
|    13|[{572, 4.7091365}...|
|    15|[{572, 4.822405},...|
|    16|[{3382, 5.1794386...|
|    17|[{572, 5.448823},...|
+------+--------------------+



In [60]:
recommendations = recommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

In [62]:
ratings.join(movies, on='MovieID').filter('UserId = 50').sort('Rating', ascending=False).limit(10).show()

AnalysisException: ignored